In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
import pandas as pd
import matplotlib.pyplot as plt # for making figures
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from pprint import pprint
import re
# import torch_xla
# import torch_xla.core.xla_model as xm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Sample data loading (replace this with any dataset URL)

filepath = '//kaggle/input/warpeace/warpeace.txt'
# Load the text
with open(filepath, 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# Preprocess the text
text = text.lower()  # Convert to lowercase
text = re.sub(r'[^a-z\s\.]', '', text)  # Remove special characters except full stops
words = text.split()  # Split into words
words = [word for word in words if word]  # Remove empty strings

# Remove words with less than 2 characters (for this example)
words = [word for word in words if len(word) > 1]
print(len(words))
words = pd.Series(words)

In [ ]:
# Build vocabulary
unique_words = sorted(set(words))
stoi = {s: i + 1 for i, s in enumerate(unique_words)}  # Map words to indices
stoi['.'] = 0  # End-of-sentence token
itos = {i: s for s, i in stoi.items()}

# Prepare input-output pairs
block_size = 5  # Number of words to use as context
X, Y = [], []

for i in range(len(words) - block_size):
    context = [0] * block_size  # Start with the end-of-sentence token
    for j in range(block_size):
        context[j] = stoi[words[i + j]]
    X.append(context)
    Y.append(stoi[words[i + block_size]])# The next word to predict

print(len(X))
print(len(Y))
print(itos[Y[-2]])

In [ ]:
X = torch.tensor(X, dtype=torch.long)  # Input tensor
Y = torch.tensor(Y, dtype=torch.long)  # Output tensor

In [ ]:
X

In [ ]:
# Define model parameters
emb_dim = 64  # Larger embedding size for words
hidden_size = 1024  # Larger hidden layer size

class NextWord(nn.Module):
    def __init__(self, block_size, vocab_size, emb_dim, hidden_size):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lin1 = nn.Linear(block_size * emb_dim, hidden_size)
        self.lin2 = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        x = self.emb(x)
        x = x.view(x.shape[0], -1)  # Flatten the embeddings
        x = F.relu(self.lin1(x))  # Use ReLU activation
        x = self.lin2(x)
        return x

# Initialize the model
model = NextWord(block_size, len(stoi), emb_dim, hidden_size).to(device)
model = model.to(device)

In [ ]:
def generate_word(model, itos, stoi, block_size, max_len=10):
    context = [0] * block_size  # Start with end-of-sentence token
    words_generated = []
    
    for _ in range(max_len):
        x = torch.tensor(context).view(1, -1).to(device)
        y_pred = model(x)
        ix = torch.distributions.categorical.Categorical(logits=y_pred).sample().item()
        word = itos[ix]
        if word == '.':
            break
        words_generated.append(word)
        context = context[1:] + [ix]  # Update context

    return ' '.join(words_generated)

In [ ]:
# loss_fn = nn.CrossEntropyLoss()
# opt = torch.optim.AdamW(model.parameters(), lr=0.01)

# # Mini-batch training
# batch_size = 13486
# num_epochs = 1000  # You can train for more epochs
# print_every = 100  # Print every 10 mini-batches

# for epoch in range(num_epochs):
#     # Shuffle data at the beginning of each epoch
#     permutation = torch.randperm(len(X))
#     X = X[permutation]
#     Y = Y[permutation]

#     for i in range(0, len(X) - (len(X)%batch_size), batch_size):
#         x = X[i:i + batch_size].to(device)
#         y = Y[i:i + batch_size].to(device)
        
#         # Forward pass
#         y_pred = model(x)
#         print('Done')
        
#         # Compute loss
#         loss = loss_fn(y_pred, y)
        
#         # Backpropagation
#         loss.backward()
        
#         # Gradient clipping
#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
#         # Update weights
#         opt.step()
#         opt.zero_grad()

#         # Print loss periodically
#         if i % print_every == 0:
#             print(f"Epoch [{epoch}], Loss: {loss.item():.4f}")
    
#     # Print loss after each epoch
#     print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")


In [ ]:

# # torch.save(model, 'model.pth')
# # torch.save(model.state_dict(), '/kaggle/working/next_word_model.pth')
# # print("Model weights saved to /kaggle/working/next_word_model.pth")
# # Function to preprocess the input sentence
# def preprocess_input(sentence, stoi, block_size):
#     sentence = sentence.lower()  # Convert to lowercase
#     sentence = re.sub(r'[^a-z\s\.]', '', sentence)  # Remove special characters except full stops
#     words = sentence.split()  # Split into words
#     words = [word for word in words if word]  # Remove empty strings

#     # Create context from the last block_size words
#     context = [0] * block_size  # Start with the end-of-sentence token
#     for i in range(block_size):
#         if i < len(words):
#             word = words[i]
#             context[i] = stoi.get(word, 0)  # Get the index, use 0 if word not in vocab
#         else:
#             context[i] = 0  # Fill with end-of-sentence token if fewer words

#     return context

# # Input sentence
# input_sentence = "can we really say that"

# # Preprocess the input sentence
# context = preprocess_input(input_sentence, stoi, block_size)

# # Convert context to tensor and send to device
# context_tensor = torch.tensor(context, dtype=torch.long).view(1, -1).to(device)

# # Generate the next word
# with torch.no_grad():  # Disable gradient calculation for inference
#     y_pred = model(context_tensor)  # Forward pass
#     ix = torch.distributions.categorical.Categorical(logits=y_pred).sample().item()  # Sample from the distribution
#     next_word = itos[ix]  # Convert index to word

# print(f"The predicted next word for the input '{input_sentence}' is: '{next_word}'")

In [ ]:
try:
    mcopy = torch.load('/kaggle/input/next-word-predictor/pytorch/default/1/model.pth')
except:
    mcopy = torch.load('/kaggle/input/next-word-predictor/pytorch/default/1/model.pth',map_location=device)
mcopy.eval()
# Function to preprocess the input sentence
def preprocess_input(sentence, stoi, block_size):
    sentence = sentence.lower()  # Convert to lowercase
    sentence = re.sub(r'[^a-z\s\.]', '', sentence)  # Remove special characters except full stops
    words = sentence.split()  # Split into words
    words = [word for word in words if word]  # Remove empty strings

    # Create context from the last block_size words
    context = [0] * block_size  # Start with the end-of-sentence token
    for i in range(block_size):
        if i < len(words):
            word = words[i]
            context[i] = stoi.get(word, 0)  # Get the index, use 0 if word not in vocab
        else:
            context[i] = 0  # Fill with end-of-sentence token if fewer words

    return context

# Input sentence
input_sentence = "men fought very bravely but"

# Preprocess the input sentence
context = preprocess_input(input_sentence, stoi, block_size)

# Convert context to tensor and send to device
context_tensor = torch.tensor(context, dtype=torch.long).view(1, -1).to(device)

# Generate the next word
with torch.no_grad():  # Disable gradient calculation for inference
    y_pred = mcopy(context_tensor)  # Forward pass
    ix = torch.distributions.categorical.Categorical(logits=y_pred).sample().item()  # Sample from the distribution
    next_word = itos[ix]  # Convert index to word

print(f"The predicted next word for the input '{input_sentence}' is: '{next_word}'")

In [ ]:
from sklearn.manifold import TSNE
# Extract word embeddings from the trained model
def extract_embeddings(model, vocab_size):
    print('Reached Function')
    with torch.no_grad():
        embeddings = model.emb.weight.cpu().numpy()  # Get the embeddings from the model
    return embeddings

# Get the word embeddings from the trained model
embeddings = extract_embeddings(model, len(stoi))

# Apply t-SNE to reduce the dimensionality of embeddings for visualization
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
print('Calling fit-transform')
embeddings_2d = tsne.fit_transform(embeddings)
print('fit-transform Done')

# Plot the t-SNE results
plt.figure(figsize=(12, 10))
print('initialing plotting')
for i, word in enumerate(itos.values()):
    print(f'Plotting{i}th point')
    plt.scatter(embeddings_2d[i, 0], embeddings_2d[i, 1], marker='o', color='red')
    plt.text(embeddings_2d[i, 0] + 0.01, embeddings_2d[i, 1] + 0.01, word, fontsize=9)
print('Done plotting')
plt.title("t-SNE visualization of word embeddings")
plt.xlabel("t-SNE component 1")
plt.ylabel("t-SNE component 2")
plt.grid(True)
plt.show()

In [ ]:
# Plot the t-SNE results
plt.figure(figsize=(12, 10))
print('initialing plotting')
for i, word in enumerate(itos.values()):
    if i == 1000:
        break
    plt.scatter(embeddings_2d[i, 0], embeddings_2d[i, 1], marker='o', color='red')
    plt.text(embeddings_2d[i, 0] + 0.01, embeddings_2d[i, 1] + 0.01, word, fontsize=9)
print('Done plotting')
plt.title("t-SNE visualization of word embeddings")
plt.xlabel("t-SNE component 1")
plt.ylabel("t-SNE component 2")
plt.grid(True)
plt.show()

In [ ]:
pip install streamlit

In [ ]:
import streamlit as st
st.title("Next Word Prediction App")
input_text = st.text_input("Enter initial text for prediction:", "I want to have")
context_length = st.selectbox("Context Length", [5,10])
embedding_dim = st.selectbox("Embedding Dimension", [8,64,256])
hidden_size = st.selectbox("Hidden Size", [1024,2048])
activation_func_name = st.selectbox("Activation Function", ["ReLU", "Tanh"])
max_len = st.selectbox("Number of words to predict", [1,5])
random_seed = st.number_input("Random Seed", min_value=0, max_value=100, value=42)
if st.button("Generate Text"):
    generated_text = generate_text(model, itos, stoi, context_length, input_text, max_len=num_words, device='cpu')
    st.write("Generated text:", input_text + " " + generated_text)




In [ ]:
!streamlit run /opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py